In [1]:
import sys
sys.version

'3.8.12 | packaged by conda-forge | (default, Sep 29 2021, 19:52:28) \n[GCC 9.4.0]'

In [2]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm   
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-71jm_g80/kobert-tokenizer_4de0280c0eba48a98440efdea3d74e88
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-71jm_g80/kobert-tokenizer_4de0280c0eba48a98440efdea3d74e88
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import gc

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")
gc.collect()
torch.cuda.empty_cache()
CUDA_LAUNCH_BLOCKING=1

/home/jihwan/anaconda3/lib/python3.8/site-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '
2023-05-22 12:43:01.552541: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [5]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
#vocab = tokenizer.get_vocab()
tok = tokenizer.tokenize

#model, vocab = BertModel.from_pretrained('skt/kobert-base-v1', tokenizer.vocab_file)

print(vocab)

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


In [6]:
data = pd.read_excel('/home/jihwan/MinJeong Archive/감정분류데이터셋.xlsx')

In [7]:
data.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
2855,혹시 유산인 가능성도 잇나요...?,공포,NaN,NaN,NaN,NaN,NaN
16680,무슨 은행이여 시발,분노,NaN,NaN,NaN,NaN,NaN
35985,사진찍어서 숫자 파악하는거 봐서 저 경찰청장도 나 앉아야 겟구만. ㅎㅎㅎ,혐오,NaN,NaN,NaN,NaN,NaN
7340,다들 그만 두셧남...?,놀람,NaN,NaN,NaN,NaN,NaN
36064,노무현 대통령과 당신은 많이 다르다,혐오,NaN,NaN,NaN,NaN,NaN
31329,재미따 재미따 재미따 ㅎㅎㅎㅎ,행복,NaN,NaN,NaN,NaN,NaN
37001,언제 터질지 모르는거 옆에 있으면.,혐오,NaN,NaN,NaN,NaN,NaN
19957,거기에 집대출까지 끼면 고충이 크겠지.,슬픔,NaN,NaN,NaN,NaN,NaN
19347,난 고영태가 부럽다...,슬픔,NaN,NaN,NaN,NaN,NaN
18338,너무 안가리면 욕먹을까봐 좀가렸더니 더욕먹었네요ㅜ,슬픔,NaN,NaN,NaN,NaN,NaN


In [8]:
data.loc[(data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
data.loc[(data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
data.loc[(data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
data.loc[(data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
data.loc[(data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
data.loc[(data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
data.loc[(data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for ques, label in zip(data['Sentence'], data['Emotion'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [9]:
class BERTSentenceTransform:
    """BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab 

    def __call__(self, line):

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')



In [10]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [11]:
'''class BERTDataset(Dataset):
    #def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len, pad, pair):
    def __init__(self, dataset, sent_idx, label_idx, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.sentences = [tokenizer.encode_plus(i[sent_idx], max_length=max_len, padding='max_length', truncation=True) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (torch.tensor(self.sentences[i]['input_ids']), torch.tensor(self.sentences[i]['attention_mask']), self.labels[i])
    
    def __len__(self):
        return (len(self.labels))
'''

"class BERTDataset(Dataset):\n    #def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len, pad, pair):\n    def __init__(self, dataset, sent_idx, label_idx, tokenizer, max_len):\n        self.tokenizer = tokenizer\n        self.sentences = [tokenizer.encode_plus(i[sent_idx], max_length=max_len, padding='max_length', truncation=True) for i in dataset]\n        self.labels = [np.int32(i[label_idx]) for i in dataset]\n\n    def __getitem__(self, i):\n        return (torch.tensor(self.sentences[i]['input_ids']), torch.tensor(self.sentences[i]['attention_mask']), self.labels[i])\n    \n    def __len__(self):\n        return (len(self.labels))\n"

In [12]:
# Setting parameters
max_len = 64
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [14]:

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

#저는 csv 파일을 사용하여 이부분은 dataset_train.values 로 해주었습니다. 
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)


In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
        #train

In [18]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [19]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/tmp/ipykernel_5495/1756086221.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1930 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.087026834487915 train acc 0.125
epoch 1 batch id 201 loss 2.009521961212158 train acc 0.1837686567164179
epoch 1 batch id 401 loss 1.7191959619522095 train acc 0.24594763092269326
epoch 1 batch id 601 loss 1.336506962776184 train acc 0.3011647254575707
epoch 1 batch id 801 loss 1.4594199657440186 train acc 0.33224094881398253
epoch 1 batch id 1001 loss 1.3453142642974854 train acc 0.36126373626373626
epoch 1 batch id 1201 loss 1.5038388967514038 train acc 0.37848667776852624
epoch 1 batch id 1401 loss 1.0467135906219482 train acc 0.3957887223411849
epoch 1 batch id 1601 loss 1.2182133197784424 train acc 0.40517645221736415
epoch 1 batch id 1801 loss 1.145581603050232 train acc 0.41390199888950585
epoch 1 train acc 0.4193741168158267


/tmp/ipykernel_5495/1756086221.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/483 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5444395149364094


  0%|          | 0/1930 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.833455502986908 train acc 0.6875
epoch 5 batch id 201 loss 0.6113192439079285 train acc 0.8274253731343284
epoch 5 batch id 401 loss 0.41248688101768494 train acc 0.8271508728179551
epoch 5 batch id 601 loss 0.7295897006988525 train acc 0.8338186356073212
epoch 5 batch id 801 loss 0.6163555383682251 train acc 0.8347378277153558
epoch 5 batch id 1001 loss 0.25137859582901 train acc 0.8408466533466533
epoch 5 batch id 1201 loss 0.28127631545066833 train acc 0.8449729392173189
epoch 5 batch id 1401 loss 0.4076000452041626 train acc 0.8480995717344754
epoch 5 batch id 1601 loss 0.2557951509952545 train acc 0.8497813866333541
epoch 5 batch id 1801 loss 0.0762912780046463 train acc 0.8501179900055524
epoch 5 train acc 0.851233513895431


  0%|          | 0/483 [00:00<?, ?it/s]

epoch 5 test acc 0.5458629103815439


In [20]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")


In [21]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 오늘은 정말 기분이 좋아요!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 이런 일이 또 생기면 화가 나요!
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 이런 일을 절대로 다시 경험하고 싶지 않아요!
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 어둠 속에서 무서운 소리가 들려와서 겁이 났어요.
>> 입력하신 내용에서 공포가 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘 날씨가 좋네요
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 이별이 너무 슬퍼서 마음이 아파요
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 정말 놀랐어요! 이런 일이 일어날 줄은 몰랐어요.
>> 입력하신 내용에서 놀람이 느껴집니다.


하고싶은 말을 입력해주세요 : 안녕하세요, 반갑습니다!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 화장품 추천 좀 해주세요
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 이런 일이 벌어질 줄이야... 너무 속상해요
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 도와주세요! 긴급 상황입니다!
>> 입력하신 내용에서 공포가 느껴집니다.


하고싶은 말을 입력해주세요 : 잘 지내셨나요?
>> 입력하신 내용에서 놀람이 느껴집니다.


하고싶은 말을 입력해주세요 : 잘 지내셨어요?
>> 입력하신 내용에서 놀람이 느껴집니다.


하고싶은 말을 입력해주세요 : 화가 나서 너무 화가 나요!
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 이 영화 정말 재미있어요!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 이 사진 정말 예쁘네요. 감사합니다!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 어떤 책을 추천